The goal of this notebook is to dump the dots csv file (with the trials' info) into an hdf5 file compatible with the dotsDB module.

The steps are as follows:
1. load data
2. loop through subjects, probCP, coherence, subjects' answers, presenceCP, viewingDuration
3. in each iteration of the loop, write corresponding dots data to hdf5 file 

In [1]:
import pandas as pd
import numpy as np
csv_data_path = "../../data/Pilot15-19/dots_join_valid_trials.csv"
csv_data = pd.read_csv(csv_data_path)
db_filename = '../../data/Pilot15-19/pilot15-19.h5'

In [2]:
import pprint

In [3]:
import sys
sys.path.append("../../motion_energy_Adrian/Python/modules/dots_db/dotsDB/")
import dotsDB as ddb

In [4]:
csv_data['viewingDuration'].iloc[100000] == 0.2

True

In [5]:
csv_data['pilotID'].equals(csv_data['i.pilotID'])

True

In [6]:
csv_data['taskID'].equals(csv_data['i.taskID'])

True

In [7]:
csv_data.shape[0]/csv_data[csv_data['coherence']==0].shape[0]

3.955917816147391

In [8]:
int(0.1*1000)

100

In [9]:
pprint.pprint(list(csv_data.columns), width=10)

['xpos',
 'ypos',
 'isActive',
 'isCoherent',
 'frameIdx',
 'seqDumpTime',
 'pilotID',
 'taskID',
 'i.taskID',
 'trialIndex',
 'trialStart',
 'trialEnd',
 'RT',
 'choice',
 'correct',
 'initDirection',
 'endDirection',
 'presenceCP',
 'coherence',
 'viewingDuration',
 'probCP',
 'timeCP',
 'randSeedBase',
 'fixationOn',
 'fixationStart',
 'targetOn',
 'dotsOn',
 'dotsOff',
 'choiceTime',
 'i.pilotID',
 'trialInSession',
 'trialClass',
 'validTrialCount',
 'signedCoherence']


In [10]:
def snowDots2DotsDBNormalizedFrames(df):
    """
    Casts a pandas.DataFrame corresponding to the .csv file outputted by some snow-dots programs into a list of 
    lists of frames, as dotsDB accepts them. The main mapping is the swapping of x and y.
    
    In dotsDB, x represents vertical position, y horizontal, and (0,0) is top left corner.
    In snow-dots, x represents horizontal position, y vertical, and (0,0) is top left corner.
      
    :param df: dataframe with columns "xpos", "ypos", "isActive", "isCoherent", "frameIdx", 
         "seqDumpTime", "pilotID", "taskID" 
    :type df: pandas.DataFrame
    """
    list_of_trials = []
    trial_indices = df["validTrialCount"].unique()
    num_trials = trial_indices.size
    
    for tr in range(num_trials):
        list_of_frames = []
        trial_data = df[df["validTrialCount"] == trial_indices[tr]]
        
        frame_indices = trial_data["frameIdx"].unique()
        num_frames = frame_indices.size
        
        assert not np.isnan(num_frames), f'trial {tr+1}, num_frames is {num_frames}'
        
        for fr in range(num_frames):
            frame_data = trial_data[(trial_data["frameIdx"] == frame_indices[fr]) & (trial_data["isActive"] == 1)]
            list_of_frames.append(np.array(frame_data[['ypos','xpos']]))  # here I swap xpos with ypos
        
        list_of_trials.append(list_of_frames)
    return list_of_trials

In [11]:
parameters = dict(speed=5, density=90, coh_mean=0, coh_stdev=10, num_frames=10, diameter=5, direction='left', # the direction parameter is 'fake' here
                 pixels_per_degree=55.4612, dot_size_in_pxs=6)
stimulus = ddb.DotsStimulus(**parameters)    

for subject in [15, 16, 17, 18, 19]:
    for prob_cp in [0.1, 0.5]:
        for ans in ['left', 'right']:
            for pres_cp in ['yes', 'no']:
                for vd in [.1, .2, .3, .4]:
                    data = csv_data[(csv_data["pilotID"] == subject) &
                                    (csv_data["probCP"] == prob_cp) &
                                    (csv_data["choice"] == ans) &
                                    (csv_data["presenceCP"] == pres_cp) & 
                                    (csv_data["viewingDuration"] == vd) & 
                                    (csv_data["coherence"] == 0)]
                    
                    ddb_data = snowDots2DotsDBNormalizedFrames(data)
                    n_trials=len(ddb_data)
                    
                    group_name = '/subj' + str(subject) + \
                                 '/probCP' + str(prob_cp) + \
                                 '/coh0' + \
                                 '/ans' + ans + \
                                 '/CP' + pres_cp + \
                                 '/VD' + str(int(vd * 1000))
                    
                    if subject == 15 and prob_cp == 0.1 and ans == 'left' and pres_cp == 'yes' and vd == .1:
                        create_f = True
                    else:
                        create_f = False
                                         
                    
                    
                    ddb.write_stimulus_to_file(stimulus, 
                                               n_trials, 
                                               db_filename, 
                                               create_file=create_f, 
                                               group_name=group_name,
                                               pre_generated_stimulus=ddb_data)

In [12]:
ddb.inspect_db(db_filename)

{'subj15': {'type': 'group', 'attrs': []},
 'subj15/probCP0.1': {'type': 'group', 'attrs': []},
 'subj15/probCP0.1/coh0': {'type': 'group', 'attrs': []},
 'subj15/probCP0.1/coh0/ansleft': {'type': 'group', 'attrs': []},
 'subj15/probCP0.1/coh0/ansleft/CPno': {'type': 'group', 'attrs': []},
 'subj15/probCP0.1/coh0/ansleft/CPno/VD100': {'type': 'group',
  'attrs': [('coh_mean', 0),
   ('coh_stdev', 10),
   ('density', 90),
   ('diameter', 5),
   ('direction', 'left'),
   ('dot_size_in_pxs', 6),
   ('field_scale', 1.1),
   ('frame_rate', 60),
   ('frame_width_in_pxs', 305),
   ('interleaves', 3),
   ('limit_life_time', True),
   ('num_frames', 20),
   ('pixels_per_degree', 55.4612),
   ('speed', 5),
   ('stencil_radius_in_vis_angle', 2.5)]},
 'subj15/probCP0.1/coh0/ansleft/CPno/VD100/px': {'type': 'dataset',
  'attrs': [],
  'shape': (7,)},
 'subj15/probCP0.1/coh0/ansleft/CPno/VD200': {'type': 'group',
  'attrs': [('coh_mean', 0),
   ('coh_stdev', 10),
   ('density', 90),
   ('diameter', 